# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


In [7]:
img_size=128

### Read validation data

In [8]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=2000

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy


In [9]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

## Read data

In [10]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [46]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202106*')]
w0=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w0)

interactive(children=(Dropdown(description='x', options=('20210614_142639_cgan2d_new_code', '20210614_131205_c…

In [47]:
result=w0.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210614_142639_cgan2d_new_code


## Plot Losses

In [48]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [49]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
23859,23859.0,152.0,0.319954,0.096151,0.416105,3.052766,85.048218,81.995453,-2.723491,85.143837,-2.035590,NaN,NaN,1.749446,-3.985702,-2.901077,0.193351,0.0002,0.0002
23860,23860.0,152.0,0.146187,0.270058,0.416245,3.792522,88.160194,84.367676,-2.696240,85.176323,-2.852299,NaN,NaN,2.691314,-1.961089,-3.748528,0.193573,0.0002,0.0002
23861,23861.0,152.0,0.221458,0.189749,0.411207,2.637635,86.215088,83.577454,-2.780648,85.197731,-2.911759,NaN,NaN,2.589451,-3.269307,-2.497524,0.193227,0.0002,0.0002
23862,23862.0,152.0,0.193657,0.262445,0.456102,4.304398,87.233421,82.929024,-2.855781,85.116936,-2.938176,NaN,NaN,3.209328,-1.946365,-4.278107,0.193227,0.0002,0.0002
23863,23863.0,152.0,0.218133,0.090529,0.308661,3.068236,86.654602,83.586365,-2.691033,85.128723,-2.904366,NaN,NaN,2.337171,-3.184452,-2.935087,0.193460,0.0002,0.0002
23864,23864.0,152.0,0.179783,0.290111,0.469894,4.168883,84.069969,79.901085,-2.614394,84.992126,-2.675324,NaN,NaN,2.846156,-2.021894,-4.113673,0.193538,0.0002,0.0002
23865,23865.0,152.0,0.302084,0.276441,0.578525,2.305208,85.593567,83.288361,-2.679410,84.583351,-2.926008,NaN,NaN,1.896362,-3.520091,-2.142801,0.193185,0.0002,0.0002
23866,23866.0,152.0,0.218037,0.396632,0.614669,5.823503,86.918625,81.095123,-2.844951,84.163757,-2.509871,NaN,NaN,4.105272,-1.155055,-5.814530,0.193267,0.0002,0.0002
23867,23867.0,152.0,0.632050,0.217694,0.849744,1.337603,79.493469,78.155869,-2.858915,82.025040,-2.561124,NaN,NaN,0.569124,-4.707923,-0.819263,0.193143,0.0002,0.0002
23868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.278519,-2.663982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [51]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.5413530349731444


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
22208,22208.0,142.0,0.302089,0.244568,0.546657,3.407223,85.006210,81.598984,-2.913563,83.886864,-3.059909,NaN,NaN,2.687975,-2.974919,-3.357495,0.193749,0.0002,0.0002
23387,23387.0,149.0,1.137761,0.241826,1.379587,1.435491,81.927185,80.491692,-2.946347,84.035950,-3.015076,NaN,NaN,-0.637815,-6.564262,-0.901875,0.193259,0.0002,0.0002
23386,23386.0,149.0,0.172876,0.864637,1.037513,8.029071,88.778473,80.749405,-2.818929,84.270493,-3.010687,NaN,NaN,4.785534,-0.092750,-8.028399,0.193073,0.0002,0.0002
22238,22238.0,142.0,0.278560,0.150290,0.428850,2.092468,80.921219,78.828751,-2.790099,81.929703,-3.008858,NaN,NaN,2.177519,-3.132290,-1.859737,0.193319,0.0002,0.0002
22210,22210.0,142.0,0.197944,0.262026,0.459971,4.133031,83.842346,79.709312,-2.796954,84.348412,-2.998958,NaN,NaN,3.396229,-2.029717,-4.084364,0.193316,0.0002,0.0002
22234,22234.0,142.0,0.414081,0.244219,0.658300,2.555132,84.993683,82.438553,-2.289821,83.857468,-2.996736,NaN,NaN,1.522813,-3.160520,-2.419426,0.193368,0.0002,0.0002
23286,23286.0,149.0,0.377517,0.163995,0.541512,2.574738,81.068893,78.494156,-2.869721,84.576157,-2.992860,NaN,NaN,1.207402,-4.299401,-2.434438,0.193628,0.0002,0.0002
22788,22788.0,146.0,0.171507,0.847309,1.018816,8.288944,88.430481,80.141541,-2.660097,84.700684,-2.989578,NaN,NaN,5.099525,0.088357,-8.287624,0.211132,0.0002,0.0002
19629,19629.0,125.0,0.280976,0.167580,0.448556,3.373222,82.464043,79.090820,-2.766175,83.464676,-2.986893,NaN,NaN,2.588190,-2.733021,-3.297650,0.194231,0.0002,0.0002
22296,22296.0,142.0,0.211268,0.193481,0.404749,2.936647,82.715591,79.778946,-2.525830,82.606956,-2.986735,NaN,NaN,2.333142,-2.636295,-2.848453,0.193319,0.0002,0.0002


In [52]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

In [53]:
# # bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

# bins=f_transform(bins)   ### scale to (-1,1)
# # ### Extract validation data
# sigma_list=[0.5,0.65,0.8,1.1]
# bkgnd_dict={}
# num_bkgnd=2000

# for label in labels_lst:
#     fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
#     print(fname)
#     samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
#     dict_val=post.f_compute_hist_spect(samples,bins)
#     bkgnd_dict[str(sigma_lst[label])]=dict_val
#     del samples

## Read stored chi-squares for images

In [82]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [83]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [90]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1b']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


### Slice best steps

In [91]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''
    
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1b']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [92]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [94]:
df_sliced=w.result
# df_sliced

In [97]:
best_step=[]
cut_off=1.0
best_step.append(f_slice_merged_df(df_merged,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[12460 14600 15190 16690 18600 21490 22040 22670 22710 23270]


array([12460, 14600, 15190, 16690, 18600, 21490, 22040, 22670, 22710,
       23270])

In [98]:
# best_step=[6176]
# best_step= [32300, 35810, 36020, 37030, 38640, 42480, 43850]

# best_step=np.arange(40130,40135).astype(int)

In [99]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 61)
[(79, 12460), (93, 14600), (97, 15190), (106, 16690), (119, 18600), (137, 21490), (141, 22040), (145, 22670), (145, 22710), (149, 23270)]


In [100]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1b
1245,79-12460,0.103435,2.514757,0.032199
1459,93-14600,0.087756,2.254461,0.030869
1518,97-15190,0.104443,2.935265,0.034151
1668,106-16690,0.106482,2.917855,0.029227
1859,119-18600,0.130146,2.471549,0.028647
2148,137-21490,0.143491,0.825256,0.033647
2203,141-22040,0.105625,1.817225,0.030290
2266,145-22670,0.139666,0.771359,0.033647
2270,145-22710,0.140980,0.827097,0.041685
2326,149-23270,0.950881,0.790019,0.862352


In [101]:
df_best

,epoch,step,img_type,fname_0,chi_1a_0,chi_1b_0,chi_1c_0,chi_1_0,chi_2_0,chi_imgvar_0,...,num_large_2,num_vlarge_2,hist_val_2,hist_err_2,hist_bin_centers_2,spec_val_2,spec_sdev_2,sum_chi_1,sum_chi_spec1,sum_chi_1b
1245,79,12460,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.008416,0.0,0.0,0.008416,0.000022,5.613109,...,0,0,"[0.6907507901357277, 0.03231582637177372, 0.02...","[0.0005915599680891856, 0.0019041933423285837,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[111519.18785767257, 81494.46730133293, 56757....","[70749.59218735575, 41754.58533958427, 23710.6...",0.103435,2.514757,0.032199
1459,93,14600,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.006343,0.0,0.0,0.006343,0.000013,6.143020,...,0,0,"[0.6920512204345077, 0.02632369991513348, 0.02...","[0.0005234449462082101, 0.001685947950415078, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[94465.18562209606, 72372.50602915286, 49289.4...","[58037.645000072465, 37731.04239532771, 22110....",0.087756,2.254461,0.030869
1518,97,15190,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.011670,0.0,0.0,0.011670,0.000047,4.689828,...,0,0,"[0.691236114995693, 0.029287719686466207, 0.02...","[0.0005822204044093344, 0.0018582528816980936,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[101411.65561444312, 77141.8538668543, 54713.7...","[61150.12587420994, 40081.59681017513, 26423.5...",0.104443,2.935265,0.034151
1668,106,16690,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.008441,0.0,0.0,0.008441,0.000012,4.759198,...,0,0,"[0.6921139722045723, 0.027172851525299066, 0.0...","[0.00045266483592140287, 0.0013993698133977753...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[91160.31426248327, 71405.57726551447, 53327.4...","[52148.44720615821, 32033.118343672366, 20689....",0.106482,2.917855,0.029227
1859,119,18600,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.016247,0.0,0.0,0.016247,0.000030,3.200811,...,0,0,"[0.6904170041247467, 0.03191928859155488, 0.02...","[0.0005261964102104003, 0.0016035240347215265,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[96419.55771376193, 74353.75886284413, 56508.4...","[53969.44647860854, 33081.43918026894, 20647.1...",0.130146,2.471549,0.028647
2148,137,21490,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.008387,0.0,0.0,0.008387,0.000008,4.826033,...,0,0,"[0.6921106343444622, 0.02760143276250529, 0.02...","[0.00043802616894955996, 0.0014197780009485884...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[106285.47976896167, 78256.82484278083, 51143....","[66538.88045582562, 35725.35037097387, 20715.0...",0.143491,0.825256,0.033647
2203,141,22040,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.008047,0.0,0.0,0.008047,0.000008,5.328899,...,0,0,"[0.6917254452877903, 0.028094100913686276, 0.0...","[0.00046454288842526714, 0.0015131812079703499...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[89054.97488182783, 70567.46559760737, 48179.0...","[59349.325770314696, 35665.346191867306, 21301...",0.105625,1.817225,0.030290
2266,145,22670,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.015096,0.0,0.0,0.015096,0.000029,4.260872,...,0,0,"[0.6919577603514333, 0.027301025353248586, 0.0...","[0.00044201308459734107, 0.001423805768557956,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[100789.15943007916, 71630.70419614017, 48258....","[62130.23848781428, 32685.646436643598, 21536....",0.139666,0.771359,0.033647
2270,145,22710,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.010014,0.0,0.0,0.010014,0.000012,4.174032,...,0,0,"[0.6911646847893435, 0.030309104877938975, 0.0...","[0.0004776086445164475, 0.0016171407484398604,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[114539.32179379463, 80159.1626075876, 53516.8...","[72007.20282887749, 38459.66441176063, 24429.8...",0.140980,0.827097,0.041685
2326,149,23270,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.015330,0.0,0.0,0.015330,0.0

### Interactive plot

In [102]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']

                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)
                plt.legend()

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:]
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [103]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type)>

In [31]:
# ### Check deterministic
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# epoch,step=0,230
# f1='20210113_185116_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210113_092234_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# epoch,step=3,20
# f1='20210114_191648_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210114_193009_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# a1=np.load(main_dir+f1)
# a2=np.load(main_dir+f2)
# # print(a1.shape,a2.shape)

# print(np.mean(a1),np.mean(a2))
# print(np.max(a1),np.max(a2))

### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [129]:
# fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
fldr=result_dir
print(fldr)
flist=glob.glob(fldr+'/models/checkpoint_*.tar')
len(flist)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_063709_cgan_predict_1.1_m2


1767

In [1]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
# #             os.remove(i)
#             pass
#         else: 
#             print(step)
#             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [128]:
best_step

array([23310, 25620, 27150, 28060, 31170, 31310, 32550, 35100, 35710,
       35740])